In [11]:
import os
from PIL import Image, ImageOps
import math 
import PyPDF2

import math 
def calculate_booklet_page_order(total_pages):
    pages_per_sheet = 8  # 4 A6 pages per side on an A4 sheet
    N = math.ceil(total_pages / pages_per_sheet) # Amount of A4's needed 
    N_total = N*pages_per_sheet
    page_order = []

   
    for i in range(N):
    # Initialize the pages for the back and front
        back = [None] * (pages_per_sheet // 2)
        front = [None] * (pages_per_sheet // 2)

        a = 1
        b = a + N*2
        c =  (N_total-2)
        d = N_total -(N+1)*2

        f = N*2 
        g = 0
        h = d+1 
        j = N_total-1

        # Fill in the page numbers for the front side
        front[0] = a + (N-i-1)*2 if total_pages - (i * 4 + 3) > 0 else None # top left
        front[1] = b + (N-i-1)*2 if total_pages - (i * 4 + 4) > 0 else None # top right
        front[2] = c - (N-i-1)*2 if i * 4 + 3 <= total_pages else None # bottom left
        front[3] = d -(N-i-1)*2 if i * 4 + 4 <= total_pages else None # bottom right 

        # Fill in the page numbers for the back side
        back[2] = g + (N-i-1)*2 if i * 4 + 2 <= total_pages else None   # top left
        back[3] = f +  (N-i-1)*2 if i * 4 + 1 <= total_pages else None # top right
        back[0] = j-(N-i-1)*2 if total_pages - (i * 4 + 1) > 0 else None # bottom left
        back[1] = h - (N-i-1)*2  if total_pages - (i * 4 + 2) > 0 else None # bottom right

       
        page_order.append((front, back))
    return page_order

def merge_pdfs(pdf_files, output_filename):
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    with open(output_filename, 'wb') as f:
        pdf_merger.write(f)
    pdf_merger.close()



In [19]:
def mm_to_pixels(mm, dpi):
    # Convert mm to pixels based on DPI (dots per inch)
    inches = mm / 25.4  # Convert mm to inches
    return int(inches * dpi)

def resize_image(image, quadrant_width, scale_factor=0.9):
    # Calculate target width (90% of quadrant width)
    target_width = int(quadrant_width * scale_factor)

    # Calculate the aspect ratio of the image
    aspect_ratio = image.width / image.height

    # New dimensions
    new_width = target_width
    new_height = int(new_width / aspect_ratio)

    return image.resize((new_width, new_height), Image.ANTIALIAS)

def create_booklet(folder_path, output_path):
    dpi = 300  # Standard print resolution
    a4_size_pixels = (mm_to_pixels(210, dpi), mm_to_pixels(297, dpi))  # A4 size in pixels

    # Load image paths
    input_paths = {}
    for f in os.listdir(folder_path):
        if f.lower().endswith('.png') and os.path.isfile(os.path.join(folder_path, f)):
            try:
                file_index = int(os.path.splitext(f)[0])
                input_paths[file_index] = os.path.join(folder_path, f)
            except ValueError:
                continue

    input_paths = dict(sorted(input_paths.items()))

    # Calculate the order of the pages in the booklet
    booklet_page_order = calculate_booklet_page_order(len(input_paths))
    pdf_files = []

    for sheet_num, (front_side, back_side) in enumerate(booklet_page_order):
        a4_page = Image.new('RGBA', a4_size_pixels, (255, 255, 255, 0))

        for side_num, side in enumerate([front_side, back_side]):
            for idx, page_num in enumerate(side):
                if page_num is None or page_num not in input_paths:
                    continue

                with Image.open(input_paths[page_num]) as img:
                    
                    #Calculate the target size (90% of quadrant size)
                    quadrant_width = a4_size_pixels[0] / 2
                    target_size = (int(quadrant_width * 0.9), int(a4_size_pixels[1] / 2 * 0.9))

                    # Resize the image
                    img1 = img.resize(target_size, Image.ANTIALIAS)

                    # Calculate quadrant center
                    quadrant_center_x = (idx % 2) * quadrant_width + quadrant_width / 2
                    quadrant_center_y = (idx // 2) * a4_size_pixels[1] / 2 + a4_size_pixels[1] / 4

                    # Calculate top-left position to paste the image
                    x = quadrant_center_x - img1.width // 2
                    y = quadrant_center_y - img1.height // 2

                    a4_page.paste(img1, (x, y))

            # Save each A4 image to a separate PDF file
            side_label = 'front' if side_num == 0 else 'back'
            output_file_name = os.path.join(output_path, f"{sheet_num+1}_{side_label}.pdf")
            a4_page.convert('RGB').save(output_file_name)
            pdf_files.append(output_file_name)

    # Merge all the PDFs into a single file
    merge_pdfs(pdf_files, os.path.join(output_path, 'booklet.pdf'))




In [18]:
img = Image.open('Input/1.png')
#mm_to_pixels((210/4)*0.9, 300)
current = img.size
#img1 = img.resize((int(current[0]*0.9), int(current[1]*0.9)))
img1 = ImageOps.scale(img, 0.9)
print(current,img1.size)
#img1.show()

(397, 559) (357, 503)


In [20]:
from functions import *

# Usage
folder_path = 'Input'  # Update with the correct folder path
output_path = 'Output'  # Output PNG file path

create_booklet(folder_path, output_path)